This is notebook gives a quick overview of this WaveNet implementation, i.e. creating the model and the data set, training the model and generating samples from it.

In [20]:
import torch
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *

## Model
This is an implementation of WaveNet as it was described in the original paper (https://arxiv.org/abs/1609.03499). Each layer looks like this:

```
            |----------------------------------------|      *residual*
            |                                        |
            |    |-- conv -- tanh --|                |
 -> dilate -|----|                  * ----|-- 1x1 -- + -->  *input*
                 |-- conv -- sigm --|     |
                                         1x1
                                          |
 ---------------------------------------> + ------------->  *skip*
```

Each layer dilates the input by a factor of two. After each block the dilation is reset and start from one. You can define the number of layers in each block (``layers``) and the number of blocks (``blocks``). The blocks are followed by two 1x1 convolutions and a softmax output function.
Because of the dilation operation, the independent output for multiple successive samples can be calculated efficiently. With ``output_length``, you can define the number these outputs. Empirically, it seems that a large number of skip channels is required.

In [21]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

In [96]:
model = WaveNetModel(layers=10,
                     blocks=4,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=256,
                     end_channels=256,
                     classes=256,
                     output_length=20,
                     kernel_size=2,
                     dtype=dtype, 
                     bias=True)
# model = load_latest_model_from('snapshots', use_cuda=use_cuda)

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (1): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (2): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (3): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (4): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (5): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (6): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (7): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (8): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (9): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (10): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (11): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (12): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (13): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (14): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (15): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (16): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (1

## Data Set
To create the data set, you have to specify a path to a data set file. If this file already exists it will be used, if not it will be generated. If you want to generate the data set file (a ``.npz`` file), you have to specify the directory (``file_location``) in which all the audio files you want to use are located. The attribute ``target_length`` specifies the number of successive samples are used as a target and corresponds to the output length of the model. The ``item_length`` defines the number of samples in each item of the dataset and should always be ``model.receptive_field + model.output_length - 1``.

```
          |----receptive_field----|
                                |--output_length--|
example:  | | | | | | | | | | | | | | | | | | | | |
target:                           | | | | | | | | | |  
```
To create a test set, you should define a ``test_stride``. Then each ``test_stride``th item will be assigned to the test set.

In [97]:
data = WavenetDataset(dataset_file='train_samples/bach_chaconne/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples/bach_chaconne',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')

one hot input
the dataset has 478569 items


In [98]:
data = WavenetDataset(dataset_file='train_samples/bach_chaconne/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples/bach_chaconne',
                      test_stride=500,
                      sampling_rate=100)
print('the dataset has ' + str(len(data)) + ' items')

one hot input
the dataset has 478569 items


In [99]:
dl = torch.utils.data.DataLoader(data)

In [100]:
sample = next(iter(dl))

In [101]:
sample[0].shape

# INPUTS

# 256 is currently set as num_classes
# 4112 is the sequence length for this input

torch.Size([1, 256, 4112])

In [104]:
sample[1].shape

# TARGETS

# 20 is currently set as the output sequence length

# i was assuming we'd have a different target for each time step
# so the dimensions would match the sequence length
# but that isn't happening

# perhaps has to do with the mu_law quantization?

torch.Size([1, 1, 20])

In [103]:
sample[1]

tensor([[[127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127,
          127, 127, 127, 127, 127, 127]]])

In [55]:
len(data)

435064

In [42]:
data[0][0].shape # one_hot

torch.Size([256, 4114])

In [51]:
data[0][0][:,0].sum() # column vectors

tensor(1.)

In [59]:
data[3][0].shape

torch.Size([256, 4114])

In [31]:
data[0][1].shape # target

torch.Size([1, 22])

In [32]:
data[0][1]

tensor([[127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127,
         127, 127, 127, 127, 127, 127, 127, 127]])

In [61]:
data[3][1].shape

torch.Size([1, 22])

In [62]:
data[0]

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127,
          127, 127, 127, 127, 127, 127, 127, 127]]))

In [76]:
out = model(next(iter(dl))[0])

In [72]:
out.shape

torch.Size([22, 256])

In [84]:
next(iter(dl))[1].shape

torch.Size([1, 1, 22])

In [73]:
next(iter(dl))[0].shape

torch.Size([1, 256, 4114])

## Training and Logging
This implementation supports logging with TensorBoard (you need to have TensorFlow installed). You can even generate audio samples from the current snapshot of the model during training. This will happen in a background thread on the cpu, so it will not interfere with the actual training but will be rather slow. If you don't have TensorFlow, you can use the standard logger that will print out to the console.
The trainer uses Adam as default optimizer.

In [8]:
def generate_and_log_samples(step):
    sample_length=32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")


logger = TensorboardLogger(log_interval=200,
                           validation_interval=400,
                           generate_interval=1000,
                           generate_function=generate_and_log_samples,
                           log_dir="logs/chaconne_model")

# logger = Logger(log_interval=200,
#                 validation_interval=400,
#                 generate_interval=1000)

In [9]:
trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         logger=logger,
                         dtype=dtype,
                         ltype=ltype)

print('start training...')
trainer.train(batch_size=16,
              epochs=10)

start training...
epoch 0


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


KeyboardInterrupt: 

## Generating
This model has the Fast Wavenet Generation Algorithm (https://arxiv.org/abs/1611.09482) implemented. This might run faster on the cpu. You can give some starting data (of at least the length of receptive field) or let the model generate from zero. In my experience, a temperature between 0.5 and 1.0 yields the best results, but this may depend on the data set.

In [6]:
start_data = data[250000][0] # use start data from the data set
start_data = torch.max(start_data, 0)[1] # convert one hot vectors to integers

def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

generated = model.generate_fast(num_samples=160000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)

/Users/lukerohrer/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/Users/lukerohrer/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0% generated
0% generated
1% generated
1% generated
one generating step does take approximately 0.014707601070404053 seconds)
2% generated
3% generated
3% generated
4% generated
4% generated
5% generated
6% generated
6% generated
7% generated
7% generated


KeyboardInterrupt: 

In [7]:
import IPython.display as ipd

ipd.Audio(generated, rate=16000)

NameError: name 'generated' is not defined